In [71]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
import random

In [5]:
class net1(nn.Module):
    def __init__(self):
        super(net1, self).__init__()
        self.layer1 = nn.Linear(225*225*3, 1000)
        self.relu   = nn.ReLU(inplace=True) 
    
    def forward(self, x):
        """
            x est le vecteur d'entrée
        """
        y = self.layer1(x)
        y = self.relu(y)
        return y

In [28]:
alex = models.alexnet(pretrained=True)

In [31]:
print(alex)

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 1000)
  )
)


In [23]:
import PIL.Image as Image
im = Image.open("/video/CLICIDE/10A-0.JPG")

In [27]:
t = torch.Tensor(1, 3, 225, 225)
trans = transforms.ToTensor()
t[0] = trans(im.resize( (225, 225) ))

In [29]:
output = alex(Variable(t))

In [32]:
output.max(1)

(Variable containing:
  9.5034
 [torch.FloatTensor of size 1x1], Variable containing:
  669
 [torch.LongTensor of size 1x1])

In [33]:
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x



In [47]:
alex464 = models.AlexNet(464)

AlexNet (
  (features): Sequential (
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU (inplace)
    (2): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU (inplace)
    (5): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU (inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU (inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU (inplace)
    (12): MaxPool2d (size=(3, 3), stride=(2, 2), dilation=(1, 1))
  )
  (classifier): Sequential (
    (0): Dropout (p = 0.5)
    (1): Linear (9216 -> 4096)
    (2): ReLU (inplace)
    (3): Dropout (p = 0.5)
    (4): Linear (4096 -> 4096)
    (5): ReLU (inplace)
    (6): Linear (4096 -> 464)
  )
)

In [54]:
for c in alex.classifier:
    print(type(c))

<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.dropout.Dropout'>
<class 'torch.nn.modules.linear.Linear'>
<class 'torch.nn.modules.activation.ReLU'>
<class 'torch.nn.modules.linear.Linear'>


In [60]:
def copyParameters(net, netBase):
    for i, f in enumerate(net.features):
        if type(f) is torch.nn.modules.conv.Conv2d:
            f.weight.data = netBase.features[i].weight.data
            f.bias.data = netBase.features[i].bias.data
    for i, c in enumerate(net.classifier):
        if type(c) is torch.nn.modules.linear.Linear:
            if c.weight.size() == netBase.classifier[i].weight.size():
                c.weight.data = netBase.classifier[i].weight.data
                c.bias.data = netBase.classifier[i].bias.data

In [56]:
copyParameters(alex464, models.alexnet(pretrained=True))

In [63]:
import torch.optim as optim
criterion = nn.loss.CrossEntropyLoss()
optimizer = optim.SGD(alex464.parameters(), lr=0.01, momentum=0.9)

In [116]:
import os.path as path
import glob
def readTrainingSet(folder="."):
    """
        Lit un dossier contenant des images
        Retourne la liste d'image avec leur classe
    """
    matchFunc = lambda x: x.split('/')[-1].split('-')[0]
    
    exts = ('*.jpg', '*.JPG', '*.JPEG', "*.png")
    r = []
    for ext in exts:
        r.extend( [(im, matchFunc(im)) for im in glob.iglob(path.join(folder, ext)) if not 'wall' in im] )
    return r

In [117]:
trainset = readTrainingSet("/video/CLICIDE/")

In [118]:
listLabel = [t[1] for t in trainset if not 'wall' in t[1]]

In [119]:
s = set(listLabel)
s = list(s)
print(len(s))

464


In [ ]:
batchSize = 32
alex464
trans = transforms.ToTensor()
for epoch in range(10):
    """
        On parcourt l'ensemble du training set
    """
    alex464.train()
    running_loss = 0.0
    random.shuffle(trainset)    
    for i in range(len(trainset)/batchSize):
        """
         1. Charge batchSize images
         2. Backprop
        """
        inputs = torch.Tensor(batchSize, 3, 225, 225)
        for j in range(batchSize):
            inputs[j] = trans(Image.open(trainset[i*batchSize+j][0]).resize( (225, 225) ))
        inputs = Variable(inputs)
        
        lab = Variable(torch.LongTensor([s.index(trainset[i*batchSize+j][1]) for j in range(batchSize)]))
        optimizer.zero_grad()
        
        outputs = alex464(inputs)
        loss = criterion(outputs, lab)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.data[0]
        if i % 10 == 9: # print every 10 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch+1, i+1, running_loss / 10))
            running_loss = 0.0
        
        
        
    """
        Eval
    """

[1,    10] loss: 6.143
[1,    20] loss: 6.084
[1,    30] loss: 6.041
[1,    40] loss: 5.915
[1,    50] loss: 5.707
[1,    60] loss: 5.503
[1,    70] loss: 5.481
[1,    80] loss: 5.295
[1,    90] loss: 5.075
[1,   100] loss: 4.737
[2,    10] loss: 3.990
[2,    20] loss: 4.320
[2,    30] loss: 4.300
[2,    40] loss: 3.964
[2,    50] loss: 4.021
[2,    60] loss: 3.972
[2,    70] loss: 4.121
[2,    80] loss: 4.176
[2,    90] loss: 4.125
[2,   100] loss: 3.864
[3,    10] loss: 2.850
[3,    20] loss: 3.330
[3,    30] loss: 3.021
[3,    40] loss: 3.085
[3,    50] loss: 3.290
[3,    60] loss: 3.062
[3,    70] loss: 2.940
[3,    80] loss: 2.637
[3,    90] loss: 2.795
[3,   100] loss: 2.678
[4,    10] loss: 1.977
[4,    70] loss: 1.932
[4,    80] loss: 2.296
[4,    90] loss: 2.013
[4,   100] loss: 2.111
[5,    10] loss: 1.562
[5,    20] loss: 1.551
[5,    30] loss: 1.635
[5,    40] loss: 1.625
[5,    50] loss: 1.590
[5,    60] loss: 1.645
[5,    70] loss: 1.597
[5,    80] loss: 1.866
[5,    90] 